<a href="https://colab.research.google.com/github/harrymkwn/InfluenceAnalysis/blob/master/AllEmotionGen.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from gensim.models import Word2Vec
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier
from gensim.models import KeyedVectors
from sklearn.model_selection import train_test_split
from google.colab import  drive
drive.mount('/content/drive')
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
device_name = tf.test.gpu_device_name()
print(device_name)
df_train = pd.read_csv('/content/drive/MyDrive/InfluenceAnalysis/Newdata/train_clean.csv',names=['Tweet','Emotion'])
df_test = pd.read_csv('/content/drive/MyDrive/InfluenceAnalysis/Newdata/test_clean.csv',names=['Tweet','Emotion'])
df_train = df_train.sample(frac=1)
df_test = df_train.sample(frac=1)

Mounted at /content/drive
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
/device:GPU:0


In [ ]:
df_train.head(1000)

,Tweet,Emotion
4706,i got all my hair cut off it almost abov my ear,1
789,janesas_mommi oh well i hope she get better,2
11371,nooanoth headachehop not a migrain,2
9166,at work boo,0
2039,smallchou I am still work,0
...,...,...
5162,my sister in law hous car and tree got hit by ...,2
4036,yay it firday have a good one everyon on my wa...,1
6221,offici out of my apt learn some stuff about th...,1
9068,fishie haha itll be gross by the time it come ...,1


In [ ]:
df_test.Emotion.unique()

array(['2', '0', '1', '3', 'emotion'], dtype=object)

In [ ]:
df_train.nunique()

Tweet      12370
Emotion        5
dtype: int64

In [ ]:
category_dict = {'0':[1,0,0,0],'1':[0,1,0,0],'2':[0,0,1,0],'3':[0,0,0,1],'emotion':[1,0,0,0]}
train_data_tweet = [x.lower().split() for x in df_train['Tweet']]
train_data_cat = np.array([category_dict[x] for x in df_train['Emotion']])
test_data_tweet = [x.lower().split() for x in df_test['Tweet']]
test_data_cat = np.array([category_dict[x] for x in df_test['Emotion']])

data_tweet = train_data_tweet + test_data_tweet

In [ ]:
# Parameters
Min_count = 0
Embedding_size = 100
Window_size = 5
Negative_sampling = 00

In [ ]:
w2v_sg = Word2Vec(min_count=Min_count,
                     window=Window_size,
                     size=Embedding_size,
                     negative=Negative_sampling,sg=1)
w2v_sg.build_vocab(data_tweet)
w2v_sg.train(data_tweet, total_examples=w2v_sg.corpus_count, epochs=5)
w2v_cbow = Word2Vec(min_count=Min_count,
                     window=Window_size,
                     size=Embedding_size,
                     negative=Negative_sampling,sg=0)
w2v_cbow.build_vocab(data_tweet)
w2v_cbow.train(data_tweet, total_examples=w2v_cbow.corpus_count, epochs=5)

(1303920, 1724630)

In [ ]:
vocab_sg = w2v_sg.wv.vocab
vocab_sg = [x for x in vocab_sg]
vocab_cbow = w2v_cbow.wv.vocab
vocab_cbow = [x for x in vocab_cbow] 
vocab_glove = {}
with open("/content/drive/My Drive/InfluenceAnalysis/glove/glove.twitter.27B.100d.txt", 'r', encoding="utf-8") as f:
  for line in f:
      values = line.split()
      word = values[0]
      vector = np.asarray(values[1:], "float32")
      vocab_glove[word] = vector

In [ ]:
batch_size = 500
Max_input_size = max([len(x) for x in data_tweet])


In [ ]:
def vec_gen(w2v,vocab,data_tweet,data_cat,batch_size,Max_input_size,Embedding_size):
  while True:
    for k in range(int(len(data_tweet)/batch_size)):
      temp = np.array([np.array([w2v.wv.get_vector(i) for i in x if i in vocab]) for x in data_tweet[k*(batch_size):(k+1)*(batch_size)]])
      temp = np.array([np.pad(x.flatten(),(0,Max_input_size*Embedding_size-len(x.flatten()))).reshape(Max_input_size,Embedding_size) for x in temp])
      tempres = data_cat[k*(batch_size):(k+1)*(batch_size)]
      yield (temp,tempres)

def glove_gen(vocab,data_tweet,data_cat,batch_size,Max_input_size,Embedding_size=100):
  while True:
    for k in range(int(len(data_tweet)/batch_size)):
      temp = np.array([np.array([vocab[i] for i in x if i in vocab.keys()]) for x in data_tweet[k*(batch_size):(k+1)*(batch_size)]])
      temp = np.array([np.pad(x.flatten(),(0,Max_input_size*Embedding_size-len(x.flatten()))).reshape(Max_input_size,Embedding_size) for x in temp])
      tempres = data_cat[k*(batch_size):(k+1)*(batch_size)]
      yield (temp,tempres)


In [ ]:
X_train = train_data_tweet
X_test = test_data_tweet
y_train = train_data_cat
y_test = test_data_cat 
result_table = [0,0,0,0,0,0,0,0,0]

In [ ]:
epochs = 20

In [ ]:
model_sg = tf.keras.Sequential()
model_sg.add(tf.keras.Input(shape=(Max_input_size,Embedding_size)))
model_sg.add(tf.keras.layers.LSTM(50))
model_sg.add(tf.keras.layers.Dense(4, activation='softmax'))
model_sg.compile(loss='categorical_crossentropy',optimizer='adam', 
                           metrics=['accuracy'])
history = model_sg.fit_generator(generator = vec_gen(w2v_sg,vocab_sg,X_train,y_train,batch_size,Max_input_size,Embedding_size),steps_per_epoch=int(len(X_train)/batch_size),epochs=epochs)
history = model_sg.evaluate(x = vec_gen(w2v_sg,vocab_sg,X_test,y_test,batch_size,Max_input_size,Embedding_size),steps=int(len(y_test)/batch_size))
print(history)

result_table[0]=history[1]
print("SG_LSTM")


Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/20
24/24 [==============================] - 5s 228ms/step - loss: 1.2688 - accuracy: 0.5683
Epoch 2/20
24/24 [==============================] - 6s 231ms/step - loss: 1.0361 - accuracy: 0.5860
Epoch 3/20
24/24 [==============================] - 6s 233ms/step - loss: 0.9882 - accuracy: 0.5860
Epoch 4/20
24/24 [==============================] - 6s 231ms/step - loss: 0.9858 - accuracy: 0.5860
Epoch 5/20
24/24 [==============================] - 6s 233ms/step - loss: 0.9857 - accuracy: 0.5860
Epoch 6/20
24/24 [==============================] - 6s 236ms/step - loss: 0.9857 - accuracy: 0.5860
Epoch 7/20
24/24 [==============================] - 6s 236ms/step - loss: 0.9856 - accuracy: 0.5860
Epoch 8/20
24/24 [==============================] - 6s 231ms/step - loss: 0.9856 - accuracy: 0.5860
Epoch 9/20
24/24 [==============================] - 6s 229ms/step - loss: 0.9856 - accuracy: 0.5860
Epoch 10/20
24/24 [=====

In [ ]:
model_sg_bi = tf.keras.Sequential()
model_sg_bi.add(tf.keras.Input(shape=(Max_input_size,Embedding_size)))
model_sg_bi.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(50),merge_mode='concat'))
model_sg_bi.add(tf.keras.layers.Dense(4, activation='softmax'))
model_sg_bi.compile(loss='categorical_crossentropy',optimizer='adam', 
                           metrics=['accuracy'])
history = model_sg_bi.fit_generator(generator = vec_gen(w2v_sg,vocab_sg,X_train,y_train,batch_size,Max_input_size,Embedding_size),steps_per_epoch=int(len(X_train)/batch_size),epochs=epochs)
history = model_sg_bi.evaluate(x = vec_gen(w2v_sg,vocab_sg,X_test,y_test,batch_size,Max_input_size,Embedding_size),steps=int(len(y_test)/batch_size))
print(history)

result_table[1]=history[1]
print("SG_BI")

Epoch 1/20
24/24 [==============================] - 6s 229ms/step - loss: 1.2083 - accuracy: 0.5760
Epoch 2/20
24/24 [==============================] - 6s 233ms/step - loss: 1.0059 - accuracy: 0.5860
Epoch 3/20
24/24 [==============================] - 6s 231ms/step - loss: 0.9875 - accuracy: 0.5860
Epoch 4/20
24/24 [==============================] - 6s 230ms/step - loss: 0.9864 - accuracy: 0.5860
Epoch 5/20
24/24 [==============================] - 6s 233ms/step - loss: 0.9864 - accuracy: 0.5860
Epoch 6/20
24/24 [==============================] - 6s 233ms/step - loss: 0.9862 - accuracy: 0.5860
Epoch 7/20
24/24 [==============================] - 6s 232ms/step - loss: 0.9861 - accuracy: 0.5860
Epoch 8/20
24/24 [==============================] - 6s 233ms/step - loss: 0.9860 - accuracy: 0.5860
Epoch 9/20
24/24 [==============================] - 6s 233ms/step - loss: 0.9859 - accuracy: 0.5860
Epoch 10/20
24/24 [==============================] - 6s 230ms/step - loss: 0.9859 - accuracy: 0.5860

In [ ]:
model_sg_rnn = tf.keras.Sequential()
model_sg_rnn.add(tf.keras.Input(shape=(Max_input_size,Embedding_size)))
model_sg_rnn.add(tf.keras.layers.SimpleRNN(50))
model_sg_rnn.add(tf.keras.layers.Dense(4, activation='softmax'))
model_sg_rnn.compile(loss='categorical_crossentropy',optimizer='adam', 
                           metrics=['accuracy'])
history = model_sg_rnn.fit_generator(generator = vec_gen(w2v_sg,vocab_sg,X_train,y_train,batch_size,Max_input_size,Embedding_size),steps_per_epoch=int(len(X_train)/batch_size),epochs=epochs)
history = model_sg_rnn.evaluate(x = vec_gen(w2v_sg,vocab_sg,X_test,y_test,batch_size,Max_input_size,Embedding_size),steps=int(len(y_test)/batch_size))
print(history)

result_table[2]=history[1]
print("SG_RNN")

Epoch 1/20
24/24 [==============================] - 6s 249ms/step - loss: 1.0900 - accuracy: 0.5729
Epoch 2/20
24/24 [==============================] - 6s 253ms/step - loss: 0.9857 - accuracy: 0.5860
Epoch 3/20
24/24 [==============================] - 6s 245ms/step - loss: 0.9856 - accuracy: 0.5860
Epoch 4/20
24/24 [==============================] - 6s 243ms/step - loss: 0.9856 - accuracy: 0.5860
Epoch 5/20
24/24 [==============================] - 6s 247ms/step - loss: 0.9856 - accuracy: 0.5860
Epoch 6/20
24/24 [==============================] - 6s 252ms/step - loss: 0.9856 - accuracy: 0.5860
Epoch 7/20
24/24 [==============================] - 6s 245ms/step - loss: 0.9856 - accuracy: 0.5860
Epoch 8/20
24/24 [==============================] - 6s 246ms/step - loss: 0.9856 - accuracy: 0.5860
Epoch 9/20
24/24 [==============================] - 6s 245ms/step - loss: 0.9857 - accuracy: 0.5860
Epoch 10/20
24/24 [==============================] - 6s 243ms/step - loss: 0.9857 - accuracy: 0.5860

In [ ]:
model_cbow = tf.keras.Sequential()
model_cbow.add(tf.keras.Input(shape=(Max_input_size,Embedding_size)))
model_cbow.add(tf.keras.layers.LSTM(50))
model_cbow.add(tf.keras.layers.Dense(4, activation='softmax'))
model_cbow.compile(loss='categorical_crossentropy',optimizer='adam', 
                           metrics=['accuracy'])
history = model_cbow.fit_generator(generator = vec_gen(w2v_cbow,vocab_cbow,X_train,y_train,batch_size,Max_input_size,Embedding_size),steps_per_epoch=int(len(X_train)/batch_size),epochs=epochs)
history = model_cbow.evaluate(x = vec_gen(w2v_cbow,vocab_cbow,X_test,y_test,batch_size,Max_input_size,Embedding_size),steps=int(len(y_test)/batch_size))
print(history)

result_table[3]=history[1]
print("CBOW_LSTM")

Epoch 1/20
24/24 [==============================] - 5s 209ms/step - loss: 1.2450 - accuracy: 0.5718
Epoch 2/20
24/24 [==============================] - 5s 214ms/step - loss: 1.0285 - accuracy: 0.5860
Epoch 3/20
24/24 [==============================] - 5s 214ms/step - loss: 0.9899 - accuracy: 0.5860
Epoch 4/20
24/24 [==============================] - 5s 226ms/step - loss: 0.9867 - accuracy: 0.5860
Epoch 5/20
24/24 [==============================] - 6s 230ms/step - loss: 0.9868 - accuracy: 0.5860
Epoch 6/20
24/24 [==============================] - 5s 218ms/step - loss: 0.9870 - accuracy: 0.5860
Epoch 7/20
24/24 [==============================] - 5s 214ms/step - loss: 0.9872 - accuracy: 0.5860
Epoch 8/20
24/24 [==============================] - 5s 212ms/step - loss: 0.9873 - accuracy: 0.5860
Epoch 9/20
24/24 [==============================] - 5s 213ms/step - loss: 0.9873 - accuracy: 0.5860
Epoch 10/20
24/24 [==============================] - 5s 209ms/step - loss: 0.9873 - accuracy: 0.5860

In [ ]:
model_cbow_bi = tf.keras.Sequential()
model_cbow_bi.add(tf.keras.Input(shape=(Max_input_size,Embedding_size)))
model_cbow_bi.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(50),merge_mode='concat'))
model_cbow_bi.add(tf.keras.layers.Dense(4, activation='softmax'))
model_cbow_bi.compile(loss='categorical_crossentropy',optimizer='adam', 
                           metrics=['accuracy'])
history = model_cbow_bi.fit_generator(generator = vec_gen(w2v_cbow,vocab_cbow,X_train,y_train,batch_size,Max_input_size,Embedding_size),steps_per_epoch=int(len(X_train)/batch_size),epochs=epochs)
history = model_cbow_bi.evaluate(x = vec_gen(w2v_cbow,vocab_cbow,X_test,y_test,batch_size,Max_input_size,Embedding_size),steps=int(len(y_test)/batch_size))
print(history)

result_table[4]=history[1]
print("CBOW_BI")

Epoch 1/20
24/24 [==============================] - 5s 210ms/step - loss: 1.2190 - accuracy: 0.5725
Epoch 2/20
24/24 [==============================] - 5s 212ms/step - loss: 1.0090 - accuracy: 0.5860
Epoch 3/20
24/24 [==============================] - 5s 214ms/step - loss: 0.9871 - accuracy: 0.5860
Epoch 4/20
24/24 [==============================] - 5s 215ms/step - loss: 0.9860 - accuracy: 0.5860
Epoch 5/20
24/24 [==============================] - 5s 212ms/step - loss: 0.9862 - accuracy: 0.5860
Epoch 6/20
24/24 [==============================] - 5s 221ms/step - loss: 0.9860 - accuracy: 0.5860
Epoch 7/20
24/24 [==============================] - 5s 223ms/step - loss: 0.9859 - accuracy: 0.5860
Epoch 8/20
24/24 [==============================] - 5s 215ms/step - loss: 0.9858 - accuracy: 0.5860
Epoch 9/20
24/24 [==============================] - 5s 216ms/step - loss: 0.9857 - accuracy: 0.5860
Epoch 10/20
24/24 [==============================] - 5s 215ms/step - loss: 0.9857 - accuracy: 0.5860

In [ ]:
model_cbow_rnn = tf.keras.Sequential()
model_cbow_rnn.add(tf.keras.Input(shape=(Max_input_size,Embedding_size)))
model_cbow_rnn.add(tf.keras.layers.SimpleRNN(50))
model_cbow_rnn.add(tf.keras.layers.Dense(4, activation='softmax'))
model_cbow_rnn.compile(loss='categorical_crossentropy',optimizer='adam', 
                           metrics=['accuracy'])
history = model_cbow_rnn.fit_generator(generator = vec_gen(w2v_cbow,vocab_cbow,X_train,y_train,batch_size,Max_input_size,Embedding_size),steps_per_epoch=int(len(X_train)/batch_size),epochs=epochs)
history = model_cbow_rnn.evaluate(x = vec_gen(w2v_cbow,vocab_cbow,X_test,y_test,batch_size,Max_input_size,Embedding_size),steps=int(len(y_test)/batch_size))
print(history)

result_table[5]=history[1]
print("CBOW_RNN")

Epoch 1/20
24/24 [==============================] - 6s 230ms/step - loss: 1.1373 - accuracy: 0.5573
Epoch 2/20
24/24 [==============================] - 5s 226ms/step - loss: 0.9900 - accuracy: 0.5860
Epoch 3/20
24/24 [==============================] - 5s 226ms/step - loss: 0.9860 - accuracy: 0.5860
Epoch 4/20
24/24 [==============================] - 5s 224ms/step - loss: 0.9861 - accuracy: 0.5860
Epoch 5/20
24/24 [==============================] - 5s 226ms/step - loss: 0.9861 - accuracy: 0.5860
Epoch 6/20
24/24 [==============================] - 5s 224ms/step - loss: 0.9861 - accuracy: 0.5860
Epoch 7/20
24/24 [==============================] - 5s 226ms/step - loss: 0.9862 - accuracy: 0.5860
Epoch 8/20
24/24 [==============================] - 5s 227ms/step - loss: 0.9862 - accuracy: 0.5860
Epoch 9/20
24/24 [==============================] - 5s 226ms/step - loss: 0.9862 - accuracy: 0.5860
Epoch 10/20
24/24 [==============================] - 6s 230ms/step - loss: 0.9862 - accuracy: 0.5860

In [ ]:
epochs=40

In [ ]:
model_glove = tf.keras.Sequential()
model_glove.add(tf.keras.Input(shape=(Max_input_size,100)))
model_glove.add(tf.keras.layers.LSTM(50))
model_glove.add(tf.keras.layers.Dense(4, activation='softmax'))
model_glove.compile(loss='categorical_crossentropy',optimizer='adam', 
                           metrics=['accuracy'])
history = model_glove.fit_generator(generator = glove_gen(vocab_glove,X_train,y_train,batch_size,Max_input_size,Embedding_size=100),steps_per_epoch=int(len(X_train)/batch_size),epochs=epochs)
history = model_glove.evaluate(x = glove_gen(vocab_glove,X_test,y_test,batch_size,Max_input_size,Embedding_size=100),steps=int(len(y_test)/batch_size))
print(history)

result_table[6]=history[1]
print("GLOVE_LSTM")


Epoch 1/40
24/24 [==============================] - 1s 32ms/step - loss: 1.1850 - accuracy: 0.5857
Epoch 2/40
24/24 [==============================] - 1s 32ms/step - loss: 0.9924 - accuracy: 0.5861
Epoch 3/40
24/24 [==============================] - 1s 33ms/step - loss: 0.9731 - accuracy: 0.5861
Epoch 4/40
24/24 [==============================] - 1s 33ms/step - loss: 0.9689 - accuracy: 0.5860
Epoch 5/40
24/24 [==============================] - 1s 32ms/step - loss: 0.9646 - accuracy: 0.5918
Epoch 6/40
24/24 [==============================] - 1s 33ms/step - loss: 0.9590 - accuracy: 0.5960
Epoch 7/40
24/24 [==============================] - 1s 32ms/step - loss: 0.9517 - accuracy: 0.5978
Epoch 8/40
24/24 [==============================] - 1s 33ms/step - loss: 0.9463 - accuracy: 0.6012
Epoch 9/40
24/24 [==============================] - 1s 33ms/step - loss: 0.9401 - accuracy: 0.6037
Epoch 10/40
24/24 [==============================] - 1s 32ms/step - loss: 0.9347 - accuracy: 0.6043
Epoch 11/

In [ ]:
model_glove_bi = tf.keras.Sequential()
model_glove_bi.add(tf.keras.Input(shape=(Max_input_size,100)))
model_glove_bi.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(50),merge_mode='concat'))
model_glove_bi.add(tf.keras.layers.Dense(4, activation='softmax'))
model_glove_bi.compile(loss='categorical_crossentropy',optimizer='adam', 
                           metrics=['accuracy'])
history = model_glove_bi.fit_generator(generator = glove_gen(vocab_glove,X_train,y_train,batch_size,Max_input_size,Embedding_size),steps_per_epoch=int(len(X_train)/batch_size),epochs=epochs)
history = model_glove_bi.evaluate(x = glove_gen(vocab_glove,X_test,y_test,batch_size,Max_input_size,Embedding_size),steps=int(len(y_test)/batch_size))
print(history)

result_table[7]=history[1]
print("GLOVE_BI")

Epoch 1/40
24/24 [==============================] - 1s 36ms/step - loss: 1.1185 - accuracy: 0.5388
Epoch 2/40
24/24 [==============================] - 1s 36ms/step - loss: 0.9635 - accuracy: 0.5879
Epoch 3/40
24/24 [==============================] - 1s 37ms/step - loss: 0.9453 - accuracy: 0.5953
Epoch 4/40
24/24 [==============================] - 1s 36ms/step - loss: 0.9273 - accuracy: 0.5985
Epoch 5/40
24/24 [==============================] - 1s 36ms/step - loss: 0.9088 - accuracy: 0.6023
Epoch 6/40
24/24 [==============================] - 1s 37ms/step - loss: 0.8949 - accuracy: 0.6062
Epoch 7/40
24/24 [==============================] - 1s 35ms/step - loss: 0.8846 - accuracy: 0.6109
Epoch 8/40
24/24 [==============================] - 1s 35ms/step - loss: 0.8764 - accuracy: 0.6163
Epoch 9/40
24/24 [==============================] - 1s 37ms/step - loss: 0.8688 - accuracy: 0.6192
Epoch 10/40
24/24 [==============================] - 1s 36ms/step - loss: 0.8611 - accuracy: 0.6252
Epoch 11/

In [ ]:
model_glove_rnn = tf.keras.Sequential()
model_glove_rnn.add(tf.keras.Input(shape=(Max_input_size,100)))
model_glove_rnn.add(tf.keras.layers.SimpleRNN(50))
model_glove_rnn.add(tf.keras.layers.Dense(4, activation='softmax'))
model_glove_rnn.compile(loss='categorical_crossentropy',optimizer='adam', 
                           metrics=['accuracy'])
history = model_glove_rnn.fit_generator(generator = glove_gen(vocab_glove,X_train,y_train,batch_size,Max_input_size,Embedding_size),steps_per_epoch=int(len(X_train)/batch_size),epochs=epochs)
history = model_glove_rnn.evaluate(x = glove_gen(vocab_glove,X_test,y_test,batch_size,Max_input_size,Embedding_size),steps=int(len(y_test)/batch_size))
print(history)

result_table[8]=history[1]
print("GLOVE_RNN")

Epoch 1/40
24/24 [==============================] - 1s 46ms/step - loss: 1.1069 - accuracy: 0.5254
Epoch 2/40
24/24 [==============================] - 1s 48ms/step - loss: 0.9894 - accuracy: 0.5858
Epoch 3/40
24/24 [==============================] - 1s 47ms/step - loss: 0.9885 - accuracy: 0.5860
Epoch 4/40
24/24 [==============================] - 1s 47ms/step - loss: 0.9884 - accuracy: 0.5860
Epoch 5/40
24/24 [==============================] - 1s 47ms/step - loss: 0.9886 - accuracy: 0.5860
Epoch 6/40
24/24 [==============================] - 1s 47ms/step - loss: 0.9883 - accuracy: 0.5860
Epoch 7/40
24/24 [==============================] - 1s 47ms/step - loss: 0.9884 - accuracy: 0.5860
Epoch 8/40
24/24 [==============================] - 1s 47ms/step - loss: 0.9884 - accuracy: 0.5860
Epoch 9/40
24/24 [==============================] - 1s 47ms/step - loss: 0.9885 - accuracy: 0.5860
Epoch 10/40
24/24 [==============================] - 1s 48ms/step - loss: 0.9885 - accuracy: 0.5860
Epoch 11/

In [ ]:
print("\tLstm\t\t\tBiLstm\t\t\tRnn")
print("sg",result_table[:3])
print("cbow",result_table[3:6])
print("glove",result_table[6:9])

	Lstm			BiLstm			Rnn
sg [0.5868333578109741, 0.5868333578109741, 0.5868333578109741]
cbow [0.5868333578109741, 0.5868333578109741, 0.5868333578109741]
glove [0.70333331823349, 0.7604166865348816, 0.6132500171661377]
